# Lab 1: Experiment Tracking
## Exercise 2: Add ClearML tracking

Go to the ClearML website and make an account. Add experiment tracking to the following code.

Following these instructions for "Jupyter Notebook": https://clear.ml/docs/latest/docs/getting_started/ds/ds_first_steps#jupyter-notebook

Explore the ClearML Project & Experiment in the web UI. Go to your project, right-click the experiment, click "Details". Look at the tabs ARTIFACTS (notebook preview), CONSOLE and SCALARS.

Note that for Jupyter Notebooks you need to manually close the task at the end. `task.close()` Otherwise your experiment will stay "Running" in ClearML web UI.

In [3]:
import os
from dotenv import load_dotenv

#%pip install -q clearml nbconvert

%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml

load_dotenv()

CLEARML_API_ACCESS_KEY=os.getenv("CLEARML_API_ACCESS_KEY")
CLEARML_API_SECRET_KEY=os.getenv("CLEARML_API_SECRET_KEY")

if CLEARML_API_ACCESS_KEY is None:
    raise KeyError("CLEARML_API_ACCESS_KEY")

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml


In [4]:
import tensorflow.keras as keras
from tensorflow.keras import layers
import numpy as np
from clearml import Task, Logger
from clearml import Task

# TODO create ClearML Task here
task = Task.init(project_name='vives-mlops-workshop', task_name='exercise2')

# download the dataset
(images, labels), _ = keras.datasets.cifar10.load_data()

# there are 10 classes of images
all_classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# choose four classes (feel free to change this!)
class_names = ["bird", "cat", "deer", "dog"]
print("Class names:", class_names)

# only keep images of these classes
class_indexes = [all_classes.index(c) for c in class_names]
to_keep = np.array([l in class_indexes for l in labels])
images = images[to_keep]
labels = labels[to_keep]

# change indexes from 10 to 2 classes
labels = np.array([class_indexes.index(l) for l in labels])

# normalize pixels between 0 and 1
images = images / 255.0

# split into train and test set
split = round(len(images) * 0.8)
train_images = images[:split]
train_labels = labels[:split]
test_images = images[split:]
test_labels = labels[split:]
print("Number of train images:", len(train_images))
print("Number of test images:", len(test_images))

# ---

# create neural network
model = keras.models.Sequential()
model.add(keras.Input(shape=(32, 32, 3)))

# convolutional layers
model.add(layers.Conv2D(4, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(8, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(8, (3, 3), activation="relu"))

# add dense layers
model.add(layers.Flatten())
model.add(layers.Dense(16, activation="relu"))
model.add(layers.Dense(4, activation="softmax"))

# ---

# compile and train the model
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))

# ---

# report accuracy to ClearML
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
logger = Logger.current_logger()

for i in range(len(accuracy)):
    logger.report_scalar("training", "accuracy", accuracy[i], iteration=i)
    logger.report_scalar("training", "val_accuracy", val_accuracy[i], iteration=i)

# close ClearML task
task.close()

ClearML Task: created new task id=ce6de5cc80c947f6b19fc08ef0207955


ClearML results page: https://app.clear.ml/projects/156920b4140a41ed894eba6f84846cb0/experiments/ce6de5cc80c947f6b19fc08ef0207955/output/log
Class names: ['bird', 'cat', 'deer', 'dog']
Number of train images: 16000
Number of test images: 4000
Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3024 - loss: 1.3541 - val_accuracy: 0.3950 - val_loss: 1.2494
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4352 - loss: 1.2030 - val_accuracy: 0.4525 - val_loss: 1.1757
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4780 - loss: 1.1406 - val_accuracy: 0.4745 - val_loss: 1.1434
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5030 - loss: 1.1057 - val_accuracy: 0.4910 - val_loss: 1.1204
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5190 - loss: 1.0821 - val_accuracy: 0.4997 - val_loss: 1.1027
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/s